# 결정트리 튜닝을 위한 교차 검증과 그리드 서치

In [1]:
import pandas as pd
wine = pd.read_csv('data/wine.csv')

# 클래스 열을 타겟으로 사용하고 나머지 열은 특성 배열에 저장

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

X_sub_train, X_val, y_sub_train, y_val = train_test_split(X_train, y_train, test_size=.2, random_state=42)

print(X_sub_train.shape, X_val.shape)

(4157, 3) (1040, 3)


In [ ]:
# 결정트리 모델 


In [2]:
from sklearn.tree import DecisionTreeClassifier
dt =DecisionTreeClassifier(random_state=42)
dt.fit(X_sub_train, y_sub_train)
print(dt.score(X_sub_train, y_sub_train))
print(dt.score(X_val, y_val))

0.9971133028626413
0.864423076923077


## 교차 검증

In [ ]:
# 교차 검증
# from sklearn.model_selection import cross_validate
# cross_validate(dt, X_train,y_train)

In [3]:
from sklearn.model_selection import cross_val_score
import numpy as np 
np.mean(cross_val_score(dt,X_train,y_train))

0.855300214703487

In [ ]:
# 분류기를 지정한 교차 검증

In [4]:
# 훈련 세트를 섞은 후 10폴드 교차 검증을 수행하려면 
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state = 42) ## 10개의 validation 
cross_val_score(dt,X_train,y_train, cv=splitter )

array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])

##  결정 트리 하이퍼 파라미터 튜닝을 위한 교차검증과 그리드 서치 하이퍼 파라미터 튜닝

#### 기본 매개 변수를 사용한 결정 트리 모델에서 min_impurity_decrease매개변수의 최적값 찾기

In [5]:
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001, 0.0002,0.0003,0.0004,0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# 결정트리 클래스의 객체를 생성하자마자 바로 전달
# cv 기본값 5
# min_impurity_decrese값마다 5번의 교차검증 25개의 모델을 훈련
# n_jobs= 병렬 실행에 사용할 CPU 코어 수(기본값 1, 모든 코어 사용: -1)

gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [6]:
best_dt = gs.best_estimator_
best_dt.score(X_train,y_train) 
# best_dt

0.9615162593804117

In [7]:
# best_params_
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [8]:
pd.DataFrame(gs.cv_results_) ## cv=5 이므로 split0,1,2,3,4.  ### split0,1,2,3,4 를 더한 값 : mean_test_score => rank_test_score로 등수 표현 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_impurity_decrease,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005593,0.001096,0.000884,0.000388,0.0001,{'min_impurity_decrease': 0.0001},0.869231,0.868269,0.882579,0.867180,0.853705,0.868193,0.009154,1
1,0.004892,0.001118,0.000693,0.000047,0.0002,{'min_impurity_decrease': 0.0002},0.871154,0.863462,0.876805,0.854668,0.856593,0.864536,0.008437,5
2,0.003713,0.000349,0.000625,0.000110,0.0003,{'min_impurity_decrease': 0.0003},0.869231,0.859615,0.875842,0.850818,0.869105,0.864922,0.008745,4
3,0.003268,0.000213,0.000556,0.000046,0.0004,{'min_impurity_decrease': 0.0004},0.869231,0.863462,0.881617,0.848893,0.875842,0.867809,0.011260,2
4,0.004514,0.000933,0.000947,0.000438,0.0005,{'min_impurity_decrease': 0.0005},0.865385,0.869231,0.882579,0.849856,0.871030,0.867616,0.010570,3


In [9]:
#각 매개변수에서 사용한 교차 검증의 평균 점수
gs

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [10]:
a= gs.cv_results_['mean_test_score']
a

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [11]:
#np.argmax() - 가장 큰 값의 인덱스 추출
# 위에서 구한 인덱스를 사용해 params 키에 저장된 매개변수 출력
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

* 복잡한 매개변수 조합의 GridSearchCV

#### 결정 트리: 불순도 감소 최소량(min_impurity_decrease), 트리깊이(max_depth), 노드를 나누기 위한최소샘플(min_samples_split) 찾기

In [12]:
params = {"min_impurity_decrease": np.arange(0.0001, 0.001, 0.0001),
         'max_depth': range(5, 20, 1), 
         'min_samples_split': range(2, 100, 10)}
# 9 * 15 * 10 = 1350개의 경우의 수 * 5폴드 => 6750개의 모델 수


In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),param_grid=params,n_jobs=-1)
gs.fit(X_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [14]:
### 가장 좋은 경우
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [15]:
## 가장 좋은 점수 
gs.best_score_

0.8683865773302731

In [16]:
y_pred = gs.best_estimator_.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8615384615384616

##  랜덤 서치

# [실습]  GridSearchCV 실습 - 사용자 행동 인식 데이터세트 결정트리

* 사용자 행동 인식(Human Activity Recognition) 데이터로 동작 예측

https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones

In [ ]:
# 출처: UCI Machine Learning Repository
# 30명의 사람에게 스마트폰 센서를 장착한 뒤 사람의 동작과 관련된 여러가지 피처를 수집
# 수집된 피처세트를 기반으로 결정트리를 이용해 어떠한 동작인지 예측
# walking, walking_upstairs, walking_downstairs, stting, standing, laying

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# features.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어 있음. 이를 DataFrame으로 로드.
feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])

# 피처명 index를 제거하고, 피처명만 리스트 객체로 생성한 뒤 샘플로 10개만 추출
feature_name = feature_name_df.iloc[:, 1].values.tolist()
print('전체 피처명에서 10개만 추출:', feature_name[:10])


In [ ]:
#중복된 피처명 확인

In [ ]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [ ]:
# train feature dataset, label data set, test용 feature dataset , label data set

* 결정트리를 사용해 동작 예측 분류 수행

* 모든 매개 변수를 기본(디폴트) 값으로 설정한 후 결정트리 수행

* GridSearchCV를 이용해 max_depth값을 변화시키면서 예측 성능 확인 (2분 ~)

In [ ]:
# 깊어진 트리는 검증 데이터 세트에서는 과적합, 성능 저하

In [ ]:
# 테스트 데이터세트에서 max_depth의 변화에 따른 값 확인

In [ ]:
# max_depth, min_samples_split 매개변수를 변경하면서 GridSearchCV수행

In [ ]:
# 테스트 데이터세트에 최적 하이퍼파라미터 작용 

In [ ]:
# 각 피처별 중요도 표현